In [1]:
# -*- coding: utf-8 -*-
import os, pickle
import argparse

import numpy as np
import torch
import torchvision
import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")

from MTL_base import MTL, generic_train
import utils

In [30]:
def dataset_with_indices(cls):

    def __getitem__(self, index):
        data, target = cls.__getitem__(self, index)
        return data, target, index

    return type(cls.__name__, (cls,), {
        '__getitem__': __getitem__,
    })

In [9]:
train_dir = "/net/scratch/hanliu-shared/data/bm/train"
valid_dir = "/net/scratch/hanliu-shared/data/bm/valid"
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=None)
valid_dataset = torchvision.datasets.ImageFolder(valid_dir, transform=None)

In [31]:
make_dataset = dataset_with_indices(torchvision.datasets.ImageFolder)

In [33]:
train_dataset  = make_dataset(train_dir, transform=None)

In [37]:
train_dataset[0]

(<PIL.Image.Image image mode=RGB size=400x400 at 0x7F08B420B580>, 0, 0)

In [61]:
i2p = []
for root, _, fnames in sorted(os.walk(train_dir, followlinks=True)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        i2p.append(path)
i2p = {i:p for i, p in enumerate(i2p)}
p2i = {v.split("/")[-1]:k for k, v in i2p.items()}
        

In [62]:
p2i

{'0.jpg': 0,
 '1.jpg': 1,
 '10.jpg': 2,
 '11.jpg': 3,
 '12.jpg': 4,
 '13.jpg': 5,
 '14.jpg': 6,
 '15.jpg': 7,
 '16.jpg': 8,
 '17.jpg': 9,
 '18.jpg': 10,
 '19.jpg': 11,
 '2.jpg': 12,
 '20.jpg': 13,
 '21.jpg': 14,
 '22.jpg': 15,
 '23.jpg': 16,
 '24.jpg': 17,
 '25.jpg': 18,
 '26.jpg': 19,
 '27.jpg': 20,
 '28.jpg': 21,
 '29.jpg': 22,
 '3.jpg': 23,
 '30.jpg': 24,
 '31.jpg': 25,
 '32.jpg': 26,
 '33.jpg': 27,
 '34.jpg': 28,
 '35.jpg': 29,
 '36.jpg': 30,
 '37.jpg': 31,
 '38.jpg': 32,
 '39.jpg': 33,
 '4.jpg': 34,
 '5.jpg': 35,
 '50.jpg': 36,
 '51.jpg': 37,
 '52.jpg': 38,
 '53.jpg': 39,
 '54.jpg': 40,
 '55.jpg': 41,
 '56.jpg': 42,
 '57.jpg': 43,
 '58.jpg': 44,
 '59.jpg': 45,
 '6.jpg': 46,
 '60.jpg': 47,
 '61.jpg': 48,
 '62.jpg': 49,
 '63.jpg': 50,
 '64.jpg': 51,
 '65.jpg': 52,
 '66.jpg': 53,
 '67.jpg': 54,
 '68.jpg': 55,
 '69.jpg': 56,
 '7.jpg': 57,
 '70.jpg': 58,
 '71.jpg': 59,
 '72.jpg': 60,
 '73.jpg': 61,
 '74.jpg': 62,
 '75.jpg': 63,
 '76.jpg': 64,
 '77.jpg': 65,
 '78.jpg': 66,
 '79.jpg': 67

In [51]:
i2p

{0: '/',
 1: 'n',
 2: 'e',
 3: 't',
 4: '/',
 5: 's',
 6: 'c',
 7: 'r',
 8: 'a',
 9: 't',
 10: 'c',
 11: 'h',
 12: '/',
 13: 'h',
 14: 'a',
 15: 'n',
 16: 'l',
 17: 'i',
 18: 'u',
 19: '-',
 20: 's',
 21: 'h',
 22: 'a',
 23: 'r',
 24: 'e',
 25: 'd',
 26: '/',
 27: 'd',
 28: 'a',
 29: 't',
 30: 'a',
 31: '/',
 32: 'b',
 33: 'm',
 34: '/',
 35: 't',
 36: 'r',
 37: 'a',
 38: 'i',
 39: 'n',
 40: '/',
 41: '1',
 42: '/',
 43: '1',
 44: '8',
 45: '9',
 46: '.',
 47: 'j',
 48: 'p',
 49: 'g'}

In [28]:
print(dir(t0))

['_Image__transformer', '__array_interface__', '__class__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_close_exclusive_fp_after_loading', '_copy', '_crop', '_dump', '_ensure_mutable', '_exif', '_expand', '_get_safe_box', '_new', '_repr_png_', '_size', 'alpha_composite', 'category', 'close', 'convert', 'copy', 'crop', 'draft', 'effect_spread', 'encoderconfig', 'encoderinfo', 'entropy', 'filter', 'format', 'format_description', 'frombytes', 'getbands', 'getbbox', 'getchannel', 'getcolors', 'getdata', 'getexif', 'getextrema', 'getim', 'getpalette', 'getpixel', 'getprojection', 'height', 'histogram', 'im', 'info', 'load', 'mode', 'palett

In [29]:
t0.info

{'jfif': 257,
 'jfif_version': (1, 1),
 'dpi': (96, 96),
 'jfif_unit': 1,
 'jfif_density': (96, 96)}

In [21]:
import os
os.listdir(train_dir+"/0")

['56.jpg',
 '87.jpg',
 '51.jpg',
 '80.jpg',
 '23.jpg',
 '24.jpg',
 '58.jpg',
 '89.jpg',
 '11.jpg',
 '16.jpg',
 '64.jpg',
 '3.jpg',
 '18.jpg',
 '63.jpg',
 '4.jpg',
 '72.jpg',
 '75.jpg',
 '35.jpg',
 '32.jpg',
 '62.jpg',
 '5.jpg',
 '65.jpg',
 '2.jpg',
 '19.jpg',
 '17.jpg',
 '10.jpg',
 '25.jpg',
 '59.jpg',
 '88.jpg',
 '22.jpg',
 '50.jpg',
 '81.jpg',
 '57.jpg',
 '86.jpg',
 '33.jpg',
 '34.jpg',
 '74.jpg',
 '73.jpg',
 '39.jpg',
 '37.jpg',
 '30.jpg',
 '79.jpg',
 '70.jpg',
 '77.jpg',
 '66.jpg',
 '1.jpg',
 '61.jpg',
 '6.jpg',
 '13.jpg',
 '8.jpg',
 '68.jpg',
 '14.jpg',
 '21.jpg',
 '26.jpg',
 '28.jpg',
 '54.jpg',
 '85.jpg',
 '53.jpg',
 '82.jpg',
 '76.jpg',
 '71.jpg',
 '78.jpg',
 '31.jpg',
 '36.jpg',
 '38.jpg',
 '52.jpg',
 '83.jpg',
 '29.jpg',
 '55.jpg',
 '84.jpg',
 '27.jpg',
 '20.jpg',
 '69.jpg',
 '15.jpg',
 '12.jpg',
 '9.jpg',
 '60.jpg',
 '7.jpg',
 '67.jpg',
 '0.jpg']

In [8]:
import matplotlib.pyplot as plt

(3, 224, 224)